In [ ]:
import tensorflow as tf

import os
import numpy as np
import matplotlib.pyplot as plt
import statistics
import pickle

import math
import cv2
import numpy as np
from numpy import cos as cos
from numpy import sin as sin
from numpy import sqrt as sqrt
from numpy import arctan2 as arctan2
from matplotlib import pyplot as plt
import os
import datetime
import time
import csv

import sys
sys.path.append('../common/')

from functions import *
from tf_functions import *
from zernike_functions import *

In [ ]:
def init_param():
    ### N: pix num, p: pix size[m]
    N = 1600
    p = 7.6e-6

    ### l_ambda: wavelength
    l_ambda = 520e-9

    ### z: distance from holo to img
    z = 0.3

    return N, z, p, l_ambda

In [ ]:
# Heaviside (Unit Step) function with grad
@tf.custom_gradient
def heaviside_custom(X):
    bin_X = (tf.math.sign(X) + 1.0) / 2.0
    bin_X = tf.cast(bin_X, dtype=tf.int64)
    bin_X = tf.cast(bin_X, dtype=tf.float64)

    #Div is differentiation intermediate value
    def grad(Div):
        return Div*1; #Heaviside has no gradient, use 1.

    return bin_X,grad;

In [ ]:
# Approximate Heaviside Function using exp
def heaviside_approx(X):
    bin_X = 1 / (1+tf.math.exp(-50*(X-0.5)))
    return bin_X

In [ ]:
def heaviside(X):
    bin_X = tf.where(X>=0.5, 1.0, 0.0)
    bin_X = tf.cast(bin_X, dtype=tf.float64)
    return bin_X

In [ ]:
gpu_id = 0
print(tf.__version__)
if tf.__version__ >= "2.1.0":
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.list_physical_devices('GPU')
    tf.config.set_visible_devices(physical_devices[gpu_id], 'GPU')
    tf.config.experimental.set_memory_growth(physical_devices[gpu_id], True)
    print('set gpu_id')

In [ ]:
##### Initialization #####
### Initial Params
N, z, p, l_ambda = init_param()
k = 2*np.pi/l_ambda

### Number of Added Images
bin_num = 5

### Directory and Path for Save
dir_fn = ''
fn = 'bin_num-' + str(bin_num)
fn = fn + '_baboon'
# fn = fn + '_peppers'
out_path = PreProcess.mkdir_out_dir(dir_fn, fn, N, p, z)


##### Check Aliasing #####
CGH.check_aliasing(z, N, l_ambda, p, z)


##### Input Image #####
in_dir = '../input/'
amp_img_np = cv2.imread(in_dir + 'baboon_gray.png', cv2.IMREAD_GRAYSCALE)
# amp_img_np = cv2.imread(in_dir + 'peppers_gray.png', cv2.IMREAD_GRAYSCALE)
size = (N, N)
amp_img_np = cv2.resize(amp_img_np, size)
amp_img_np = amp_img_np / 255.0
amp_img = tf.constant(amp_img_np)
amp_img = tf.dtypes.cast(amp_img, tf.float64)


##### Initial Amplitude Hologram #####
input_vars = []
for i in range(bin_num):
    random_amp = np.random.uniform(0.0, 1.0, size)
    input_vars.append(tf.Variable(random_amp))


##### Incident Light toward SLM #####
light_img = tf.fill(size, 1.0)
light_img = tf.dtypes.cast(light_img, tf.complex128)


##### Target Image #####
source_amp = np.sqrt(amp_img)
source_img = source_amp

# When using Intensity Distribution
goal_img = amp_img
goal_img_tf = tf.constant(goal_img)
goal_img_tf = tf.dtypes.cast(goal_img_tf, tf.complex128)

target = amp_img

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
loss_list = []
psnr_list = []

def loss_func():
    sum_prop_inten = tf.zeros(size, dtype=tf.dtypes.float64)
    
    for i in range(bin_num):
        cur_amp = input_vars[i]
        bin_amp = heaviside_approx(cur_amp)
        imag = tf.zeros(bin_amp.shape, dtype=tf.dtypes.float64)
        phase_exp = tf.dtypes.complex(bin_amp, imag)
        init_plane = tf.math.multiply(light_img, phase_exp)
        
        # Propagation from SLM to Image Plane without normalization
        init_plane_add = tf_CGH.add_zero_padding(init_plane)
        N = init_plane_add.shape[0]
        prop = tf_CGH.band_limited_angular_spectrum(init_plane_add, k, N, l_ambda, z, p)
        prop_rm = tf_CGH.remove_zero_padding(prop)
        N = prop_rm.shape[0]
        
        prop_inten = prop_rm * tf.math.conj(prop_rm)
        prop_inten = tf.dtypes.cast(prop_inten, tf.float64)

        sum_prop_inten += prop_inten
    
    # Nomarilzation
    sum_prop_inten_norm = tf_CGH.normalize_amp_one(sum_prop_inten)
    
    err = sum_prop_inten_norm - target
    

    # Show Images
    imgs = [
        sum_prop_inten_norm.numpy()*255,
        target * 255,
        
        CGH.amp_abs(CGH.intensity(init_plane.numpy())),
        CGH.amp_abs(CGH.amplitude(init_plane.numpy())),
        CGH.phase_norm(CGH.phase(init_plane.numpy())),
    ]
    ImageProcess.show_imgs(imgs)

    
    # PSNR
    psnr_val = cv2.PSNR(sum_prop_inten_norm.numpy(), target.numpy(), R=1)
    psnr_list.append(psnr_val)
    print('PSNR: ', psnr_val)
    
    
    # mean square
    loss = ( err ** 2 )    
    # absolute value
    # loss = tf.math.abs(err)

    print(tf.reduce_mean(loss).numpy())
    loss_list.append(tf.reduce_mean(loss).numpy())
    
    return loss


start = time.time()

for i in range(5):
    step_count = opt.minimize(loss_func, input_vars).numpy()
    print(step_count)
    
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")


In [ ]:
##### Validate Optimization Result (0,1 only) #####
bin_amp_list = []
sum_prop_inten = tf.zeros(size, dtype=tf.dtypes.float64)
prop_inten_norm_list = []

for i in range(bin_num):
    cur_amp = input_vars[i]
    bin_amp = heaviside(cur_amp)
    bin_amp_list.append(bin_amp.numpy() * 255)

    imag = tf.zeros(bin_amp.shape, dtype=tf.dtypes.float64)
    phase_exp = tf.dtypes.complex(bin_amp, imag)
    init_plane = tf.math.multiply(light_img, phase_exp)

    # Propagation from SLM to Image Plane without normalization
    init_plane_add = tf_CGH.add_zero_padding(init_plane)
    N = init_plane_add.shape[0]
    prop = tf_CGH.band_limited_angular_spectrum(init_plane_add, k, N, l_ambda, z, p)
    prop_rm = tf_CGH.remove_zero_padding(prop)
    N = prop_rm.shape[0]

    prop_inten = prop_rm * tf.math.conj(prop_rm)
    prop_inten = tf.dtypes.cast(prop_inten, tf.float64)
    
    # Normalization
    prop_inten_norm_list.append(tf_CGH.normalize_amp_one(prop_inten).numpy()*255)

    sum_prop_inten += prop_inten

    
# Normalization
sum_prop_inten_norm = tf_CGH.normalize_amp_one(sum_prop_inten)

err = sum_prop_inten_norm - target

psnr_val = cv2.PSNR(sum_prop_inten_norm.numpy(), target.numpy(), R=1)
print('PSNR: ', psnr_val)


# Save Simulation Result
imgs = [
    sum_prop_inten_norm.numpy() * 255
]
ImageProcess.show_imgs(imgs)
opt_out_path = out_path + 'sim_'
ImageProcess.save_imgs(opt_out_path, imgs)

imgs = prop_inten_norm_list
ImageProcess.show_imgs(imgs)
opt_out_path = out_path + 'each_sim_'
ImageProcess.save_imgs(opt_out_path, imgs)


# Save Measurement Data (computation time, PSNR, loss_list)
with open(out_path+'loss_list.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows([loss_list])

with open(out_path+'psnr_list.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows([psnr_list])

with open(out_path+'PSNR.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows([[psnr_val]])

with open(out_path+'computation_time.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows([[elapsed_time]])


# Save Each Binarization Image
imgs = bin_amp_list
ImageProcess.show_imgs(imgs)
opt_out_path = out_path + 'bin_'
ImageProcess.save_imgs(opt_out_path, imgs)

# Check Histogram of Binarization Image
plt.hist(imgs[0].ravel(),256,[0,256]); plt.show()

In [ ]:
##### Validate Optimization Result (using Approximate Heaviside) #####
bin_amp_list = []
sum_prop_inten = tf.zeros(size, dtype=tf.dtypes.float64)
prop_inten_norm_list = []

for i in range(bin_num):
    cur_amp = input_vars[i]
    bin_amp = heaviside_approx(cur_amp)
    bin_amp_list.append(bin_amp.numpy() * 255)

    imag = tf.zeros(bin_amp.shape, dtype=tf.dtypes.float64)
    phase_exp = tf.dtypes.complex(bin_amp, imag)
    init_plane = tf.math.multiply(light_img, phase_exp)

    # Propagation from SLM to Image Plane without normalization
    init_plane_add = tf_CGH.add_zero_padding(init_plane)
    N = init_plane_add.shape[0]
    prop = tf_CGH.band_limited_angular_spectrum(init_plane_add, k, N, l_ambda, z, p)
    prop_rm = tf_CGH.remove_zero_padding(prop)
    N = prop_rm.shape[0]

    prop_inten = prop_rm * tf.math.conj(prop_rm)
    prop_inten = tf.dtypes.cast(prop_inten, tf.float64)
    
    # Normalization
    prop_inten_norm_list.append(tf_CGH.normalize_amp_one(prop_inten).numpy()*255)

    sum_prop_inten += prop_inten

    
# Normalization
sum_prop_inten_norm = tf_CGH.normalize_amp_one(sum_prop_inten)

err = sum_prop_inten_norm - target

psnr_val = cv2.PSNR(sum_prop_inten_norm.numpy(), target.numpy(), R=1)
print('PSNR: ', psnr_val)


# Show Simulation Result
imgs = [
    sum_prop_inten_norm.numpy() * 255
]
ImageProcess.show_imgs(imgs)

imgs = prop_inten_norm_list
ImageProcess.show_imgs(imgs)


# Show Each Binarization Image
imgs = bin_amp_list
ImageProcess.show_imgs(imgs)

# Check Histogram of Optimized Image
plt.hist(imgs[0].ravel(),256,[0,256]); plt.show()